In [5]:
# AlphaVantage API didn't have a great selection,
# but I had already written helpers for it before
# I relaized this. It might be helpful to create an
# interoperable interface to use this API later
# depending on data availability 

import requests
import json
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
if not API_KEY:
    raise Exception("Alpha vantage requires env API key as ALPHA_VANTAGE_API_KEY")

# Browse the API for valid stocks
def queryAlphaVantageStock(ticker):
    stockApiUrl = "https://www.alphavantage.co/query"
    querystring = {"function":"SYMBOL_SEARCH", "keywords":ticker, "apikey":API_KEY}

    response = requests.request("GET", stockApiUrl, params=querystring)
    if response.status_code != 200:
        raise Exception(f'Expected response to be 200, received {response}')

    bm = response.json()['bestMatches']
    if not bm:
        print('No best match')
        print(response.json())
    for i in bm:
        print(i['1. symbol'])
        print(i)

# write valid stock to json
def getAVJson(ticker):
    url = "https://www.alphavantage.co/query"
    querystring = {"function":"TIME_SERIES_MONTHLY_ADJUSTED", "symbol":ticker, "apikey":API_KEY}

    response = requests.request("GET", url, params=querystring)
    if response.status_code != 200:
        raise Exception(f'Expected response to be 200, received {response}')

    return response.json()

# parse stock json into df
def cleanAVFromJson(filename):
    with open(filename, "r") as infile:
        j = json.load(infile)

    df = pd.DataFrame(j["Monthly Adjusted Time Series"]).T
    df.columns = ["open", "high", "low", "close", "adjusted_close", "volume","dividend_amount"]
    df = df.astype({
        "open": np.float32, 
        "high": np.float32, 
        "low": np.float32,
        "close": np.float32,
        "adjusted_close": np.float32,
        "volume": np.int32,
        "dividend_amount": np.float32
    })
    
    return df


VWDLI5N0UH176O44
